<h3> Step 2 imputed missing data with mice in R</h3><br>
<h3> Step 3:Tried feature selection using PCA.</h3>

In [248]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier  
from sklearn import preprocessing 
import random 
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import statsmodels.api as sm
import dataImputation 

random.seed(123)

os.getcwd()
os.chdir('/Users/khadekirti/Desktop/Data Science')

In [249]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.005, 
                       threshold_out = 0.005, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [260]:
def getFeatures(data): 
    b = data.columns.get_loc("GDP")
    # for all columns, apart from GDP, replace negative values with zero
    columns = data.select_dtypes(include=[np.number]).columns.tolist()
    columns.remove('GDP')
    X = data[columns]
    y = data.iloc[ : , b].values
    featureSelection = stepwise_selection(X, y)
    return featureSelection

In [261]:
def fitTransform(dfAll):
    # Fill zeroes in data  
    #dfAll = dfAll.fillna(0)
    # Get column names first
    names = list(dfAll.columns)
    # List 
    names.remove('Country Code')
    names.remove('Year')
    # Data Frame to be scalarised
    scaler = preprocessing.MinMaxScaler()
    scaled_df = scaler.fit_transform(dfAll[names])
    scaled_df = pd.DataFrame(scaled_df, columns= names)
    # Add index
    extra_df = dfAll[['Country Code','Year']]
    scaled_df = scaled_df.reset_index()
    extra_df = extra_df.reset_index()
    # Merge 
    dfAll = pd.merge(extra_df,scaled_df, how = 'inner', on = ['index'])
    del dfAll['index']
    return dfAll

In [262]:
def dataProcessing(df, year):
    dfAggeregateAll = pd.DataFrame(columns = ['Country Code','Year'])
    k = 1  
    df = df[['Country Code','Indicator Code', year]] 
    listIndicatorCode = list(np.unique(df["Indicator Code"])) 
    for j in listIndicatorCode:
        df_sub = df[['Country Code','Indicator Code', year]]
        df_sub = df_sub.loc[df_sub['Indicator Code'] == j]
        df_sub['Year'] = year
        df_sub[j] = df_sub[year]
        del df_sub[year]
        del df_sub['Indicator Code']
        if (k == 1):
            dfAggeregateAll = pd.merge(dfAggeregateAll,df_sub, how = "outer", on = ['Country Code', 'Year'])
            k = 2
        else: 
            dfAggeregateAll = pd.merge(dfAggeregateAll,df_sub, how = "inner", on = ['Country Code', 'Year'])
    return dfAggeregateAll 

In [263]:
dfGDPAll = pd.DataFrame()
for year in ['2008', '2009', '2010', '2011', '2012','2013']: 
    for i in os.listdir(): 
        if ("GDP" in i):
            dfGDP = pd.read_csv( os.getcwd() + '/' + i + '/GDP.csv' , skiprows=4)
            del dfGDP['Unnamed: 63']
            dfGDP = dfGDP[['Country Code','Indicator Code', year]]
            listIndicatorCode = list(np.unique(dfGDP["Indicator Code"]))
            dfGDP['Year'] = year
            dfGDP['GDP'] = dfGDP[year]
            del dfGDP[year]
            del dfGDP['Indicator Code']
            dfGDPAll = dfGDPAll.append(dfGDP)

In [268]:
featureSelection= getFeatures(dfAllYear)

/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  AG.PRD.CREL.MT                 with p-value 1.03654e-204
Add  SP.RUR.TOTL                    with p-value 1.22011e-195
Add  NV.AGR.TOTL.CD                 with p-value 7.77985e-13
Add  AG.LND.AGRI.K2                 with p-value 6.1615e-21
Add  AG.LND.FRST.K2                 with p-value 1.39279e-50
Add  AG.LND.ARBL.HA.PC              with p-value 1.78597e-10
Add  AG.LND.CREL.HA                 with p-value 1.25966e-13
Add  AG.LND.FRST.ZS                 with p-value 0.000909308
Add  AG.LND.ARBL.ZS                 with p-value 0.000165693


In [270]:
dfAllAggregation = pd.DataFrame(columns = ['Country Code', 'Year'])
listAllColumns = ['Country Code', 'Year'] 
m = 1
for i in os.listdir(): 
    if ((".DS_Store" not in i) & (".py" not in i) & (".ipynb" not in i) & (".csv" not in i)): 
        for j in os.listdir(os.getcwd() + '/' + i):
            if (('csv' in j) & ('Metadata' not in j) & ('GDP' not in j)):
                print(j)
                df = pd.read_csv( os.getcwd() + '/' + i + '/' + j, skiprows=4)
                # If 'Unnamed: 63' in columns, delete the column 
                if ('Unnamed: 63' in  list(df.columns)): 
                    del df['Unnamed: 63']
                dfAllYear = pd.DataFrame()
                for year in ['2008', '2009', '2010', '2011', '2012','2013']: 
                    # List the Indicator Code 
                    # Process the dataframe 
                    dfAggeregateAll = dataProcessing(df,year)   
                    dfAll =  pd.merge(dfGDPAll,dfAggeregateAll, how = 'inner', on = ['Country Code', 'Year'])  
                    dfAll = dfAll.loc[:, dfAll.isnull().mean() < 0.4] 
                    # Standarise the numerical value
                    dfAll = fitTransform(dfAll)
                    dfAllYear = dfAllYear.append(dfAll)
                #Get features and 
                dfAllYear = dfAllYear.fillna(0)
                featureSelection= getFeatures(dfAllYear)
                if featureSelection != None:
                    if (len(RFEList)) > 1:
                        for i in RFEList: 
                            if ((i  not in listAllColumns)): 
                                RFEListFinal.append(i) 
                                
                    featureSelection.extend(['Country Code', 'Year','GDP'])
                    dfAllYear = dfAllYear[featureSelection] 
                    if (m == 1): 
                        dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "outer",on = ['Country Code', 'Year'])
                        m = 2
                    else:
                        del dfAll['GDP']
                        dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "inner",on = ['Country Code', 'Year']) 
                else: 
                     print("This did not have any feature to be selected: " + j)
    else: 
        pass            

Agriculture.csv


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curren

Add  AG.LND.TOTL.K2                 with p-value 0.0
Add  AG.SRF.TOTL.K2                 with p-value 6.67131e-248
Add  AG.PRD.CREL.MT                 with p-value 2.47874e-64
Add  SP.RUR.TOTL                    with p-value 9.80524e-247
Add  AG.LND.AGRI.K2                 with p-value 3.04032e-138
Add  AG.LND.FRST.K2                 with p-value 1.02172e-44
Add  AG.LND.ARBL.HA.PC              with p-value 5.74974e-26
Add  AG.LND.CREL.HA                 with p-value 1.26048e-17
Add  AG.LND.ARBL.ZS                 with p-value 4.84996e-07
Add  AG.LND.FRST.ZS                 with p-value 9.26748e-06
Add  AG.LND.CROP.ZS                 with p-value 0.00114387
Add  SP.RUR.TOTL.ZS                 with p-value 0.00496192
API_15_DS2_en_csv_v2_10587064.csv


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  SM.POP.REFG                    with p-value 4.33174e-137
Add  SP.DYN.LE00.FE.IN              with p-value 9.47741e-18
Add  SL.TLF.ACTI.FE.ZS              with p-value 2.62106e-08
Add  SL.EMP.VULN.FE.ZS              with p-value 2.13454e-15
Drop SP.DYN.LE00.FE.IN              with p-value 0.289134
Add  SL.UEM.TOTL.FE.ZS              with p-value 0.000127657


...

Add  SE.SEC.ENRL.VO                 with p-value 3.91231e-300
Add  SE.TER.ENRR.MA                 with p-value 1.38839e-18
Add  SE.PRM.AGES                    with p-value 3.11901e-06
Add  SE.SEC.ENRL.GC                 with p-value 1.39008e-05
Add  SE.SEC.ENRL                    with p-value 3.0151e-06
Add  SE.PRM.ENRL                    with p-value 1.48359e-06
Add  SL.TLF.TOTL.IN                 with p-value 0.000841932
Add  SE.PRM.TCHR                    with p-value 1.1066e-09
Drop SE.PRM.ENRL                    with p-value 0.0146469


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Add  SE.SEC.ENRL.VO.FE.ZS           with p-value 7.6021e-05
Add  SE.ENR.TERT.FM.ZS              with p-value 0.00130875
Add  SE.PRM.UNER                    with p-value 0.00176673
Add  SL.TLF.TOTL.FE.ZS              with p-value 0.00387451
API_12_DS2_en_csv_v2_10587438.csv


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


KeyboardInterrupt: 

In [271]:
dfAllAggregation.head()

,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,AG.PRD.CREL.MT,SP.RUR.TOTL,AG.LND.AGRI.K2_x,AG.LND.FRST.K2_x,AG.LND.ARBL.HA.PC,AG.LND.CREL.HA,AG.LND.ARBL.ZS,AG.LND.FRST.ZS,...,SE.PRM.AGES,SE.SEC.ENRL.GC,SE.SEC.ENRL,SL.TLF.TOTL.IN_y,SE.PRM.TCHR,SE.SEC.ENRL.VO.FE.ZS,SE.ENR.TERT.FM.ZS,SE.PRM.UNER,SL.TLF.TOTL.FE.ZS_y,GDP_y
0,0.000001,0.000001,0.000000,0.000017,3.559299e-07,1.044127e-07,0.009475,0.000000,0.185326,0.023699,...,0.5,0.000009,0.000012,0.000000,0.000019,0.502135,0.266223,0.000001,0.000000,0.000043
1,0.005126,0.004953,0.001570,0.006271,7.937175e-03,3.356121e-04,0.137920,0.003859,0.199125,0.021003,...,1.0,0.002973,0.002678,0.002903,0.004095,0.023602,0.000000,0.000000,0.503829,0.000177
2,0.009790,0.009458,0.000300,0.002720,1.207852e-02,1.460027e-02,0.075441,0.001644,0.045460,0.478472,...,0.5,0.000824,0.001214,0.002860,0.000000,0.499010,0.000000,0.015148,0.915231,0.001230
3,0.000215,0.000218,0.000242,0.000442,2.472038e-04,1.935960e-04,0.099946,0.000212,0.371366,0.288671,...,0.5,0.000789,0.000763,0.000387,0.000368,0.483848,0.249911,0.000428,0.735961,0.000172
4,0.000004,0.000004,0.000000,0.000003,4.283722e-06,3.977625e-06,0.004319,0.000000,0.026940,0.345767,...,0.5,0.000004,0.000006,0.000000,0.000014,0.753250,0.000000,0.000000,0.000000,0.000057


In [208]:
dfAggeregateAll.head()

,Country Code,Year,AG.AGR.TRAC.NO,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,...,SI.POV.RUGP,SI.POV.RUHC,SL.AGR.EMPL.FE.ZS,SL.AGR.EMPL.MA.ZS,SL.AGR.EMPL.ZS,SP.RUR.TOTL,SP.RUR.TOTL.ZG,SP.RUR.TOTL.ZS,TM.VAL.AGRI.ZS.UN,TX.VAL.AGRI.ZS.UN
0,ABW,2008,NaN,NaN,NaN,20.000000,11.111111,2.000000e+03,0.019733,11.111111,...,NaN,NaN,NaN,NaN,NaN,56978.0,0.782300,56.217,NaN,NaN
1,AFG,2008,NaN,151.977291,2.988068,379100.000000,58.067580,7.794000e+06,0.285557,11.938241,...,NaN,NaN,76.815002,50.759998,59.293999,20929063.0,2.242984,76.680,0.002371,5.033140
2,AGO,2008,NaN,NaN,8.258824,576900.000000,46.274164,3.400000e+06,0.156254,2.727200,...,21.6,58.3,52.084999,37.269001,44.585999,9077595.0,1.762792,41.718,NaN,NaN
3,ALB,2008,7438.0,NaN,75.877049,11810.000000,43.102190,6.100000e+05,0.206968,22.262774,...,2.6,14.6,56.901001,35.759998,44.655998,1473922.0,-2.921578,50.009,0.839336,7.653021
4,AND,2008,355.0,NaN,NaN,207.600002,44.170213,7.600000e+02,0.009063,1.617021,...,NaN,NaN,NaN,NaN,NaN,9221.0,4.219661,10.996,0.421743,0.804129


In [104]:
dfAllAggregation = pd.DataFrame(columns = ['Country Code', 'Year'])
listAllColumns = ['Country Code', 'Year'] 
m = 1
for i in os.listdir(): 
    if ((".DS_Store" not in i) & (".py" not in i) & (".ipynb" not in i) & (".csv" not in i)): 
        for j in os.listdir(os.getcwd() + '/' + i):
            if (('csv' in j) & ('Metadata' not in j) & ('GDP' not in j)):
                print(j)
                df = pd.read_csv( os.getcwd() + '/' + i + '/' + j, skiprows=4)
                # If 'Unnamed: 63' in columns, delete the column 
                if ('Unnamed: 63' in  list(df.columns)): 
                    del df['Unnamed: 63']
                dfAllYear = pd.DataFrame()
                for year in ['2008', '2009', '2010', '2011', '2012','2013']: 
                    # List the Indicator Code 
                    listIndicatorCode = list(np.unique(df["Indicator Code"])) 
                    # Process the dataframe 
                    dfAggeregateAll = dataProcessing(df,year)   
                    dfAll =  pd.merge(dfGDP,dfAggeregateAll, how = 'inner', on = ['Country Code', 'Year'])  
                    dfAll = dfAll.loc[:, dfAll.isnull().mean() < 0.3] 
                    # Standarise the numerical value
                    dfAll = fitTransform(dfAll)
                    dfAllYear = dfAllYear.append(dfAll)
                # Get features and 
                featureSelection= getFeatures(dfAllYear)
                if featureSelection != None:
                    featureSelection.extend(['Country Code', 'Year','GDP'])
                    dfAllYear = dfAllYear[featureSelection] 
                    if (m == 1): 
                        dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "outer",on = ['Country Code', 'Year'])
                        m = 2
                    else:
                        del dfAll['GDP']
                        dfAllAggregation = pd.merge(dfAllAggregation,dfAllYear, how = "inner",on = ['Country Code', 'Year']) 
                else: 
                     print("This did not have any feature to be selected: " + j)
    else: 
        pass

Agriculture.csv


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  NV.AGR.TOTL.ZS                 with p-value 1.83077e-10
API_15_DS2_en_csv_v2_10587064.csv


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  SE.ENR.SECO.FM.ZS              with p-value 1.73334e-16
Add  SL.EMP.VULN.FE.ZS              with p-value 5.08132e-11
Add  SL.TLF.ACTI.MA.ZS              with p-value 1.8103e-22
Add  SL.TLF.CACT.MA.ZS              with p-value 2.50391e-07
Aid Effectiveness.csv


...

/Users/khadekirti/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/Users/khadekirti/anaconda3

Add  FS.AST.CGOV.GD.ZS              with p-value 7.46424e-11
Add  FD.AST.PRVT.GD.ZS              with p-value 4.88194e-06
Add  FP.CPI.TOTL.ZG                 with p-value 0.00308823
API_16_DS2_en_csv_v2_10585316.csv


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


Add  EG.ELC.ACCS.UR.ZS              with p-value 2.89984e-16
Add  EN.ATM.PM25.MC.M3              with p-value 1.32011e-07
Add  EN.POP.EL5M.UR.ZS              with p-value 1.70632e-05
Add  EN.URB.LCTY.UR.ZS              with p-value 0.00238496
API_9_DS2_en_csv_v2_10580187.csv
Add  IT.MLT.MAIN                    with p-value 7.87343e-11
Add  IT.NET.BBND.P2                 with p-value 4.89912e-08
Add  BX.GSR.CCIS.CD                 with p-value 9.13028e-06


/Users/khadekirti/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/Users/khadekirti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.


In [273]:
dfAllAggregation

,AG.LND.TOTL.K2,AG.SRF.TOTL.K2,AG.PRD.CREL.MT,SP.RUR.TOTL,AG.LND.AGRI.K2_x,AG.LND.FRST.K2_x,AG.LND.ARBL.HA.PC,AG.LND.CREL.HA,AG.LND.ARBL.ZS,AG.LND.FRST.ZS,...,SE.PRM.AGES,SE.SEC.ENRL.GC,SE.SEC.ENRL,SL.TLF.TOTL.IN_y,SE.PRM.TCHR,SE.SEC.ENRL.VO.FE.ZS,SE.ENR.TERT.FM.ZS,SE.PRM.UNER,SL.TLF.TOTL.FE.ZS_y,GDP_y
0,1.397724e-06,1.350452e-06,0.000000e+00,0.000017,3.559299e-07,1.044127e-07,0.009475,0.000000e+00,0.185326,0.023699,...,0.5,8.855331e-06,1.213655e-05,0.000000e+00,0.000019,0.502135,0.266223,0.000001,0.000000,0.000043
1,5.126489e-03,4.953111e-03,1.569753e-03,0.006271,7.937175e-03,3.356121e-04,0.137920,3.859379e-03,0.199125,0.021003,...,1.0,2.972630e-03,2.678152e-03,2.902853e-03,0.004095,0.023602,0.000000,0.000000,0.503829,0.000177
2,9.789546e-03,9.458463e-03,2.999265e-04,0.002720,1.207852e-02,1.460027e-02,0.075441,1.644162e-03,0.045460,0.478472,...,0.5,8.243154e-04,1.213588e-03,2.860449e-03,0.000000,0.499010,0.000000,0.015148,0.915231,0.001230
3,2.151395e-04,2.181057e-04,2.418214e-04,0.000442,2.472038e-04,1.935960e-04,0.099946,2.120241e-04,0.371366,0.288671,...,0.5,7.889908e-04,7.629763e-04,3.865956e-04,0.000368,0.483848,0.249911,0.000428,0.735961,0.000172
4,3.674913e-06,3.550628e-06,0.000000e+00,0.000003,4.283722e-06,3.977625e-06,0.004319,0.000000e+00,0.026940,0.345767,...,0.5,3.951357e-06,5.707227e-06,0.000000e+00,0.000014,0.753250,0.000000,0.000000,0.000000,0.000057
5,8.822414e-02,8.554896e-02,1.624006e-02,0.045361,8.565593e-02,4.765704e-03,0.059903,2.531136e-02,0.056508,0.017330,...,0.0,5.480689e-02,5.431019e-02,3.444629e-02,0.071100,0.566847,0.194010,0.105687,0.203905,0.031137
6,5.576603e-04,6.342423e-04,9.450310e-07,0.000343,1.139604e-04,7.835424e-05,0.003557,1.521568e-07,0.012084,0.045075,...,0.5,0.000000e+00,0.000000e+00,1.528057e-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.004673
7,2.148951e-02,2.109436e-02,1.777665e-02,0.001146,3.020394e-02,7.267121e-03,0.421856,1.550645e-02,0.214912,0.108491,...,0.5,8.331718e-03,7.452113e-03,5.833383e-03,0.010397,0.000000,0.298928,0.000375,0.700873,0.006349
8,2.235415e-04,2.256166e-04,1.626113e-04,0.000317,3.660844e-04,8.238656e-05,0.074773,2.419449e-04,0.263886,0.118230,...,1.0,6.410662e-04,5.975946e-04,4.113575e-04,0.000000,0.529124,0.248674,0.000000,0.827529,0.000163
9,1.554771e-06,1.502189e-06,0.000000e+00,0.000002,9.421675e-07,4.420136e-06,0.025019,0.000000e+00,0.246867,0.902949,...,0.5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000009


In [111]:
dfAllAggregation.columns

Index(['NV.AGR.TOTL.ZS_x', 'Country Code', 'Year', 'GDP', 'SE.ENR.SECO.FM.ZS',
       'SL.EMP.VULN.FE.ZS', 'SL.TLF.ACTI.MA.ZS', 'SL.TLF.CACT.MA.ZS_x',
       'DC.DAC.JPNL.CD', 'DC.DAC.TOTL.CD', 'DT.ODA.ALLD.KD',
       'EG.ELC.ACCS.ZS_x', 'NV.AGR.TOTL.ZS_y', 'AG.LND.EL5M.UR.ZS',
       'EN.ATM.CO2E.PC', 'EN.ATM.CO2E.KD.GD_x', 'EN.ATM.NOXE.ZG',
       'SL.UEM.1524.FE.ZS', 'SL.EMP.TOTL.SP.MA.ZS', 'SL.TLF.CACT.MA.ZS_y',
       'SL.TLF.CACT.FM.ZS', 'SL.EMP.TOTL.SP.FE.ZS', 'SL.TLF.CACT.FE.ZS',
       'SL.EMP.1524.SP.FE.ZS', 'SL.TLF.ACTI.1524.FE.ZS', 'SL.EMP.TOTL.SP.ZS',
       'BM.GSR.TOTL.CD', 'BX.GSR.TOTL.CD', 'BX.KLT.DINV.CD.WD',
       'BX.GSR.TRVL.ZS', 'TM.VAL.MRCH.R5.ZS', 'TX.VAL.MRCH.R5.ZS',
       'BX.GSR.CMCP.ZS', 'TX.VAL.OTHR.ZS.WT_x', 'BX.GSR.INSF.ZS',
       'TM.VAL.OTHR.ZS.WT_x', 'SP.POP.1014.FE.5Y', 'SP.POP.1014.MA.5Y',
       'SP.DYN.TO65.MA.ZS', 'SP.POP.BRTH.MF', 'SP.POP.7074.FE.5Y',
       'SP.POP.1564.TO.ZS', 'SP.POP.1564.FE.ZS', 'SP.POP.1564.MA.ZS',
       'SP.POP.0509.FE